## Choose proper tutors for math exam

## Link: https://www.kaggle.com/c/choose-tutors/data

In [55]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [56]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

X = df_train.drop(['Id', 'choose'], axis=1)
y = df_train['choose']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.3, random_state=42)

In [ ]:
# test_df.loc[:,['Id','Credit Default']].to_csv('Try_6.csv',index=None)